In [1]:
from sympy import *
from LieAlgebras5 import *

**VECTOR SPACES**

In [2]:
V = VectorSpace()
V.basis_symbolic = ['b1', 'b2']
V.basis_symbolic

[b1, b2]

In [3]:
[b1, b2] = V.basis_symbolic
a1, a2 = symbols('a1 a2')
v_symbolic = 3*a1*b1 + a2*b2
display(v_symbolic)
v_array = V.from_symbols_to_array(v_symbolic)
display(v_array)
print(v_array)

3*a1*b1 + a2*b2

[[3*a1], [a2]]

[[3*a1], [a2]]


In [4]:
V.from_array_to_symbols(v_array) == v_symbolic

True

In [5]:
V.from_array_to_symbols([1,2])

b1 + 2*b2

In [6]:
V.from_array_to_symbols(['a','b'])

a*b1 + b*b2

In [7]:
V.a_vector_symbolic('x') 
# notice that the internal and standard indexing start with 0, 
# while in this notebook we were starting from 1

x_0*b1 + x_1*b2

In [8]:
V.growth_vector = [1,1]
V.growth_vector

[1, 1]

In [9]:
V.graded_basis_symbolic

[[b1], [b2]]

In [10]:
# Notice that, since we had already set a symbolic basis, we still have the same:
V.basis_symbolic

[b1, b2]

In [11]:
# We can, and it is better to do it, uniform the two symbolic basis:
# Notice that we don't have a method to transform expressions in the graded basis
# if we it is not uniformed to the symbolic basis.
V.basis_symbolic = flatten(V.graded_basis_symbolic)
V.basis_symbolic

[b1, b2]

In [12]:
display(V.weights)
display(v_array)
display(V.dil(2,v_array))

[1, 2]

[[3*a1], [a2]]

Matrix([
[6*a1],
[4*a2]])

In [13]:
# This is the automatic symbolic dual basis:
V.dual_basis_symbolic

[@b1, @b2]

In [14]:
# If you really want to have different symbols, you can use the inned method that builds it:
V._dual_basis_symbolic_build(pre_symbol_for_dual='(',post_symbol_for_dual=')^*')
V.dual_basis_symbolic

[(b1)^*, (b2)^*]

In [15]:
V_dual = V.dual_space
display( V_dual.basis_symbolic )

[(b1)^*, (b2)^*]

In [16]:
# notice that vector in the dual space are still vertical arrays:
display( V_dual.basis_coord )
# A way to get around this is to define an outer basis of horizontal arrays:
dim = V_dual.dimension
V_dual.basis_outer = [ Array(v, (1,dim)) for v in eye(dim).rowspace()]
display(V_dual.basis_outer)

[[[1], [0]], [[0], [1]]]

[[[1, 0]], [[0, 1]]]

In [17]:
# At this point, we can define the methods to transform things from one to the other:
V_dual.from_array_to_outer = lambda v: v.reshape(1,V_dual.dimension)
V_dual.from_outer_to_array = lambda v: v.reshape(V_dual.dimension,1)
V_dual.from_symbols_to_outer = lambda v: V_dual.from_array_to_outer( V_dual.from_symbols_to_array(v) )
V_dual.from_outer_to_symbols = lambda v: V_dual.from_array_to_symbols( V_dual.from_outer_to_array(v) )

In [18]:
# We have a method for _ordered_ duality of symbolic expressions
for a in V.basis_symbolic :
    for b in V.dual_basis_symbolic:
        print(b,a,V.pairing_dualVSvect_symbolic(b,a) )
    print()

(b1)^* b1 1
(b2)^* b1 0

(b1)^* b2 0
(b2)^* b2 1



**LINEAR MAPS**

In [19]:
L = MultLinMap()
L.domains = V,V
print(L.num_domains)
L.range = V
[b1, b2] = V.basis_symbolic
a = symbols('a')
L.rules = {(b1,b2): b1, (b2,b1): -b1}
# A multilinear map is callable. The call is just an application of the method L.apply()
# L.apply(a*3*b2 , a*b1 + b2)
L(a*3*b2 , a*b1 + b2)

2


-3*a**2*b1

In [20]:
L( Array([1,0],(2,1)) , Array([0,1],(2,1)) )

[[1], [0]]

In [21]:
L = LinMap()
# this snipped works also with 
# L = MultLinMap()
L.domains = V
print(L.num_domains)
L.range = V
[b1, b2] = V.basis_symbolic
a = symbols('a')
# if L = LinMap(), then any of the following two lines works fine.
# however, if L = MultLinMap(), then only the second one works 
L.rules = {b1:b2, b2: -b1}
# L.rules = {(b1,):b2, (b2,): -b1}
L(3*a*b1 + b2)

1


3*a*b2 - b1

In [22]:
L( Array([1,0],(2,1)) )

[[0], [1]]

**HEISENBERG ALGEBRA**

In [23]:
heis = LieAlgebra()
heis.name = 'Heisenberg'
heis.basis_symbolic = ['X','Y','Z']
[X,Y,Z] = heis.basis_symbolic
heis.brackets.rules = {(X,Y):Z}
print(heis.brackets.rules)
heis.declare_stratified(isit = True, growth_vector = [2,1], step = 2)

{(X, Y): Z, (Y, X): -Z}


In [24]:
[e1,e2,e3] = heis.basis_coord
heis(e1,e2)

[[0], [0], [1]]

In [25]:
heis.check_jacobi()

True

In [26]:
# the class heis is callable:
# it calls the methdo heis.brackets,
# which is actually an instance of MultLinMap
print(type(heis.brackets))
# Notice that the rules are made automatically antisymmetric:
print('Lie brackets are antisymmetric: ', heis.brackets.is_antisymmetric)
print(heis.brackets.rules) 
print(heis(X,Y))
print(heis(Y,X))

<class 'LieAlgebras5.MultLinMap'>
Lie brackets are antisymmetric:  True
{(X, Y): Z, (Y, X): -Z}
Z
-Z


In [27]:
# Brackets have a description as tensor:
display(heis.brackets.as_tensor)
# structure constants are derived from this tensor, which is much larger than necessary
display(heis.structure_constants)
help(heis.__class__.structure_constants)

[[[[[[0, 0, 0]], [[0, 0, 0]], [[0, 0, 0]]]]], [[[[[0, 0, 0]], [[0, 0, 0]], [[0, 0, 0]]]]], [[[[[0, 1, 0]], [[-1, 0, 0]], [[0, 0, 0]]]]]]

[[[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 1, 0], [-1, 0, 0], [0, 0, 0]]]

Help on property:

    Returns the structure constants of the Lie algebra,
    
    that is a tensor Gamma of shape (dim,dim,dim) so that
    self(b_i,b_j) = sum_k Gamma[^k,_i,_j] b_k



In [28]:
# We can declare a Lie algebra: nilpotent, or graded, or stratified. 
# The following line declares all three at the same time:
heis.declare_stratified(isit = True, growth_vector = [2,1], step = 2)
# Notice that at this point the automatic graded basis is recovered from the symbolic one:
display(heis.graded_basis_symbolic)
# If we want a graded basis with the correct indices, we can do this:
heis._graded_basis_symbolic_build(force_build = True)
display(heis.graded_basis_symbolic)
# If heis.basis_symbolic was not defined already, it would be overwritten by now.
# However, since we forced the method, heis.basis_symbolic has not been overwritten.
display(heis.basis_symbolic)
# HOWEVER, THIS CELL SEEM TO BREAK THE CONSTRUCTION OF THE JET SPACE

[[X, Y], [Z]]

[[b^0_0, b^0_1], [b^1_0]]

[X, Y, Z]

In [29]:
# THIS CELL IS TO FIX THE DAMAGE OF THE PREVIOUS ONE.
heis = LieAlgebra()
heis.name = 'Heisenberg'
heis.basis_symbolic = ['X','Y','Z']
[X,Y,Z] = heis.basis_symbolic
heis.brackets.rules = {(X,Y):Z}
heis.declare_stratified(isit = True, growth_vector = [2,1], step = 2)

In [30]:
# We have an implementation of the BCH formula, which works pretty well when the group is nilpotent.
display(heis.bch(X+Y,Y+Z))
display(heis.bch(e1+e2,e2+e3))

X + 2*Y + 3*Z/2

[[1], [2], [3/2]]

In [31]:
# there is a method that produces some technical data that are useful on Lie algebras:
print(heis._a_basis_of_brackets_build.__doc__ )
# this produces the following three objects (not very readable indeed)
display(heis.a_basis_of_brackets)
display(heis.a_basis_of_brackets_graded)
display(heis.a_basis_of_brackets_dict)


        Builds self._a_basis_of_brackets_graded , self._a_basis_of_brackets , and self._a_basis_of_brackets_dict

        that is, 
        a basis in coordinates for *self*
        out of the set of all brackets of the first layer,
        both organized in layers and flattened,
        and a dictionary that tells you for each element of the basis which are the two vectors it is a bracket of.

        Works only for stratified Lie algebras.
        


[[[1], [0], [0]], [[0], [1], [0]], [[0], [0], [1]]]

[[[[1], [0], [0]], [[0], [1], [0]]], [[[0], [0], [1]]]]

{[[0], [0], [1]]: [[[1], [0], [0]], [[0], [1], [0]]]}

In [32]:
# We also have similar methods that boild everything down to the first layer:
print(heis.__class__.a_basis_symbolic_of_brackets_dict.__doc__)
display(heis.a_basis_symbolic_of_brackets_dict)
print(heis.__class__.a_basis_symbolic_of_brackets_dict_V1.__doc__)
display(heis.a_basis_symbolic_of_brackets_dict_V1)
display(heis.boil_to_V1(Z))


Like self.a_basis_of_brackets_dict(), but using symbolic basis.
        


{Z: X*Y - Y*X}


Returns a dictionary {z: x*y-y*x}

so that z is the bracket x and y, and both x and y are in the first layer only.
So, it is like self.a_basis_symbolic_of_brackets_dict(),
but where only elements of the first layer appear
        


{Z: X*Y - Y*X}

X*Y - Y*X

**SUBALGEBRAS**

In [ ]:
heis2 = LieAlgebra()
heis2.name = 'Heisenberg 2'
heis2.basis_symbolic = ['X1','Y1','X2','Y2','Z']
[X1,Y1,X2,Y2,Z] = heis2.basis_symbolic
heis2.brackets.rules = {(X1,Y1):Z, (X2,Y2):Z}
print(heis2.brackets.rules)
heis2.declare_stratified(isit = True, growth_vector = [4,1], step = 2)

In [ ]:
W = heis2.lie_subalgebra([X1,Y1,Z],'a')
A = W.basis_symbolic
W(A[0],A[1])

**MORPHISMS, EMBEDDINGS, ISOMORPHISMS**

In [8]:
heis = LieAlgebra()
heis.dimension = 3
[X,Y,Z] = heis.basis_symbolic
heis.brackets.rules = {(X,Y):Z}

heis2 = LieAlgebra()
heis2.dimension = 5
[X1,Y1,X2,Y2,Z] = heis2.basis_symbolic
heis2.brackets.rules = {(X1,Y1):Z, (X2,Y2):Z}

W1 = heis2.lie_subalgebra([X1,Y1,Z],'a')

W2 = heis2.lie_subalgebra([X1,X2,Z],'aa')

In [12]:
L, sol = heis.morphisms_to(W1)
L.as_matrix = L.as_matrix.subs(sol[0])
print(L.as_matrix)

Matrix([[L_0_0, L_0_1, 0], [L_1_0, L_1_1, 0], [L_2_0, L_2_1, L_0_0*L_1_1 - L_0_1*L_1_0]])


In [13]:
M, sol = heis.morphisms_to(W2,smbl='M')
M.as_matrix = M.as_matrix.subs(sol)
print(M.as_matrix)

Matrix([[M_0_0, M_0_1, 0], [M_1_0, M_1_1, 0], [M_2_0, M_2_1, 0]])


In [14]:
K, sol = heis.morphisms_to(heis2,smbl='K')
K.as_matrix = K.as_matrix.subs(sol[0])
print(K.as_matrix)

Matrix([
[K_0_0, K_0_1,                                                     0],
[K_1_0, K_1_1,                                                     0],
[K_2_0, K_2_1,                                                     0],
[K_3_0, K_3_1,                                                     0],
[K_4_0, K_4_1, K_0_0*K_1_1 - K_0_1*K_1_0 + K_2_0*K_3_1 - K_2_1*K_3_0]])

**JET ALGEBRAS**

In [33]:
heisJet = JetAlgebra()
heisJet.name = 'Heisenberg Jet'
heisJet.lie_algebra_domain = heis
heisJet.order = 2
heisJet.build_me()

Step 1 of 6: Construct set of indices:
[((-1, 0, 0), 0), ((0, -1, 0), 0), ((0, 2, 0), @1), ((1, 1, 0), @1), ((0, 0, 1), @1), ((2, 0, 0), @1), ((0, 0, -1), 0), ((1, 0, 0), @1), ((0, 1, 0), @1), ((0, 0, 0), @1)]

Step 2 of 6: Construct HD basis:
{((0, 0, 0), @1): @1, ((1, 0, 0), @1): @X*@1, ((0, 1, 0), @1): @Y*@1, ((0, 2, 0), @1): @Y**2*@1, ((1, 1, 0), @1): @X*@Y*@1 + @Y*@X*@1, ((0, 0, 1), @1): -@X*@Y*@1, ((2, 0, 0), @1): @X**2*@1}

Step 3 of 6: and outer basis:
[X, Y, @Y**2*@1, @X*@Y*@1 + @Y*@X*@1, -@X*@Y*@1, @X**2*@1, Z, @X*@1, @Y*@1, @1]

Step 4 of 6: Construct sybolic basis:
[A^0_(-1, 0, 0), A^0_(0, -1, 0), A^@1_(0, 2, 0), A^@1_(1, 1, 0), A^@1_(0, 0, 1), A^@1_(2, 0, 0), A^0_(0, 0, -1), A^@1_(1, 0, 0), A^@1_(0, 1, 0), A^@1_(0, 0, 0)]

Step 5 of 6: Construct functions from outer basis to the others.
Building transformation operations: this can take a lot of time. You may want to change the source code.

Step 6 of 6: Construct Lie bracket operation

Done


In [34]:
basis = []
basis = heisJet.basis_symbolic
dim = heisJet.dimension
M = zeros(dim+1,dim+1)
for i in range(dim):
    M[i+1,0] = M[0,i+1] = basis[i]
for i in range(dim):
    for j in range(dim):
#         print(i,j)
        M[i+1,j+1] = heisJet(basis[i],basis[j])
display(M)

Matrix([
[             0,  A^0_(-1, 0, 0),  A^0_(0, -1, 0),  A^@1_(0, 2, 0),  A^@1_(1, 1, 0), A^@1_(0, 0, 1),  A^@1_(2, 0, 0), A^0_(0, 0, -1),  A^@1_(1, 0, 0),  A^@1_(0, 1, 0), A^@1_(0, 0, 0)],
[A^0_(-1, 0, 0),               0,  A^0_(0, 0, -1),               0, -A^@1_(0, 1, 0),              0, -A^@1_(1, 0, 0),              0, -A^@1_(0, 0, 0),               0,              0],
[A^0_(0, -1, 0), -A^0_(0, 0, -1),               0, -A^@1_(0, 1, 0), -A^@1_(1, 0, 0), A^@1_(1, 0, 0),               0,              0,               0, -A^@1_(0, 0, 0),              0],
[A^@1_(0, 2, 0),               0,  A^@1_(0, 1, 0),               0,               0,              0,               0,              0,               0,               0,              0],
[A^@1_(1, 1, 0),  A^@1_(0, 1, 0),  A^@1_(1, 0, 0),               0,               0,              0,               0,              0,               0,               0,              0],
[A^@1_(0, 0, 1),               0, -A^@1_(1, 0, 0),               0

In [35]:
from tabulate import tabulate
M_list = [[M[i,j] for j in range(M.cols)] for i in range(M.rows)]
tabella = tabulate(M_list,headers="firstrow", tablefmt="html")
display(tabella)

0,"A^0_(-1, 0, 0)","A^0_(0, -1, 0)","A^@1_(0, 2, 0)","A^@1_(1, 1, 0)","A^@1_(0, 0, 1)","A^@1_(2, 0, 0)","A^0_(0, 0, -1)","A^@1_(1, 0, 0)","A^@1_(0, 1, 0)","A^@1_(0, 0, 0)"
"A^0_(-1, 0, 0)",0,"A^0_(0, 0, -1)",0,"-A^@1_(0, 1, 0)",0,"-A^@1_(1, 0, 0)",0,"-A^@1_(0, 0, 0)",0,0
"A^0_(0, -1, 0)","-A^0_(0, 0, -1)",0,"-A^@1_(0, 1, 0)","-A^@1_(1, 0, 0)","A^@1_(1, 0, 0)",0,0,0,"-A^@1_(0, 0, 0)",0
"A^@1_(0, 2, 0)",0,"A^@1_(0, 1, 0)",0,0,0,0,0,0,0,0
"A^@1_(1, 1, 0)","A^@1_(0, 1, 0)","A^@1_(1, 0, 0)",0,0,0,0,0,0,0,0
"A^@1_(0, 0, 1)",0,"-A^@1_(1, 0, 0)",0,0,0,0,0,0,0,0
"A^@1_(2, 0, 0)","A^@1_(1, 0, 0)",0,0,0,0,0,0,0,0,0
"A^0_(0, 0, -1)",0,0,0,0,0,0,0,0,0,0
"A^@1_(1, 0, 0)","A^@1_(0, 0, 0)",0,0,0,0,0,0,0,0,0
"A^@1_(0, 1, 0)",0,"A^@1_(0, 0, 0)",0,0,0,0,0,0,0,0
"A^@1_(0, 0, 0)",0,0,0,0,0,0,0,0,0,0
